In [1]:
import datetime
import numpy as np
import pandas as pd
import os
import tensorflow as tf
tf.random.set_seed(777)
drop_rate=0.3

In [2]:
def min_max_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [3]:
def create_model(x,y):
    tf.model=tf.keras.Sequential()
    tf.model=tf.keras.Sequential()
    tf.model.add(tf.keras.layers.Conv1D(filters=40, kernel_size=6, input_shape=(x.shape[1],1),strides=1,activation='relu'))
    tf.model.add(tf.keras.layers.Dropout(drop_rate))
# L2
    tf.model.add(tf.keras.layers.Conv1D(filters=40, kernel_size=2,strides=1, activation='relu'))
    tf.model.add(tf.keras.layers.Dropout(drop_rate))
# L fully connected
    tf.model.add(tf.keras.layers.Flatten())
    tf.model.add(tf.keras.layers.Dense(units=y.shape[1], kernel_initializer='glorot_normal', activation='relu'))
    
    
    tf.model.compile(loss='mse', optimizer=tf.optimizers.Adam(lr=0.001), metrics=['accuracy'])
    tf.model.summary()

In [4]:
def load_model():
    number=int(np.loadtxt('min_loss_accuracy_wine_CNN.csv',delimiter=','))
    print(number)
    tf.model.load_weights(f"train_model_wine_ver2_CNN/cp-{number}.ckpt")

In [5]:
xy1=pd.read_csv('datasets_Wine.csv')
xy=xy1.values
for i in range(0,xy.shape[0]):
    if(xy[i][-1]=='good'):
        xy[i][-1]=1
    elif(xy[i][-1]=='bad'):
        xy[i][-1]=0

xy[:,:-1]=min_max_scaler(xy[:,:-1])
test_line=int(xy.shape[0]*0.7)
x_data1=xy[:,:-1]
x_data1=x_data1.reshape(x_data1.shape[0],x_data1.shape[1],1)
x_data=x_data1[:test_line,:]

y_data=xy[:test_line,[-1]] 
x_test=x_data1[test_line:,:]
y_test=xy[test_line:,[-1]] 
x_data = np.array(x_data, dtype=np.float32) 
y_data = np.array(y_data, dtype=np.float32) 
x_test = np.array(x_test, dtype=np.float32) 
y_test = np.array(y_test, dtype=np.float32) 

In [6]:
create_model(x_data,y_data)
load_model()
tf.model.summary()
evaluate = tf.model.evaluate(x_test, y_test) 
print("loss: {0}, accuracy: {1}".format(evaluate[0], evaluate[1]))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 6, 40)             280       
_________________________________________________________________
dropout (Dropout)            (None, 6, 40)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 40)             3240      
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 40)             0         
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 3,721
Trainable params: 3,721
Non-trainable params: 0
____________________________________________________